In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk

In [2]:
data= pd.read_csv('fake_or_real_news.csv')
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
data.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [4]:
data.label.value_counts()

REAL    3171
FAKE    3164
Name: label, dtype: int64

In [5]:
x=data.drop('label',axis=1)
x

,Unnamed: 0,title,text
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello..."
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T..."
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...
...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene..."


In [6]:
y= data['label']
y.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [7]:
data.shape

(6335, 4)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [9]:
messages=data.copy()

In [10]:
messages

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [11]:
msg=data.sample(n=1000)

In [20]:
msg.reset_index(inplace=True)
msg

,level_0,index,Unnamed: 0,title,text,label
0,0,1821,9818,Hillary Ally Offers Strategy On How To Make Ne...,Trump To Host Facebook Live Nightly Show Until...,FAKE
1,1,1957,5121,Picking Mike Pence Really Was A Grand Slam For...,Mike Pence? Full disclosure: I served as head ...,REAL
2,2,4249,9370,4 Times As Many Americans Think Biased U.S. Me...,"Posted on November 1, 2016 by WashingtonsBlog ...",FAKE
3,3,708,3897,Secret Service testing ways to intercept rogue...,"Mysterious, middle-of-the-night drone flights ...",REAL
4,4,1894,1937,Rubio nabs key former Romney aide,Sen. Marco Rubio (R-Fla.) is adding a veteran ...,REAL
...,...,...,...,...,...,...
995,995,4854,2801,Administration officials defend Iran nuclear deal,WASHINGTON—Top administration officials forcef...,REAL
996,996,4341,3477,Who will fight for gay marriage at SCOTUS?,Washington (CNN) The date for arguments is on ...,REAL
997,997,4071,4626,How Gary Johnson's Vote Percentages Will Affec...,There has been a lot of weight placed in Liber...,REAL
998,998,3900,1794,What advantages does Scott Walker offer?,"Every GOP presidential candidate, most especia...",REAL


In [21]:
msg.label.value_counts()

FAKE    521
REAL    479
Name: label, dtype: int64

In [22]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

In [23]:
ps= PorterStemmer()

In [24]:
corpus=[]

In [25]:
for i in range(len(msg)):
    rv= re.sub('[^a-zA-Z]',' ',msg['text'][i])
    rv= rv.lower()
    rv= rv.split()
    
    rv=[ps.stem(word) for word in rv if word not in stopwords.words('english')]
    corpus.append(" ".join(rv))
    

In [27]:
corpus

['trump host facebook live nightli show elect day opinion sulzberg high arthur pretti big wuss go lot hillari would one call tanden believ clinton also need get minor women report corner keep campaign afloat advertis stori continu also think brown women pundit shame time other social media conclud cultiv joan walsh yglesia allen perri bacon greg sargent defend help embolden fwiw push pir yr ago posit accept refer pundit anyon brown anymor mayb work schulzburg wuss endors clinton last month',
 'mike penc full disclosur serv head super pac seek draft penc presidenti race long persuad penc superior leadership qualiti even less object usual call penc conserv champion good reason penc made right decis run governor indiana opportun distinguish public servic happen also perfect boot camp vice presid honor david mcintosh presid power club growth one encourag penc come back elector polit mcintosh later serv guru draft penc presid super pac recent exclus interview mcintosh recal vacat congressio

In [28]:
len(corpus)

1000

In [30]:
cv=CountVectorizer(max_features=5000,ngram_range=(1,3))
x=cv.fit_transform(corpus).toarray()

In [31]:
x.shape

(1000, 5000)

In [46]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [81]:
tfidf= TfidfVectorizer(max_features=5000,ngram_range=(1,3))
x=tfidf.fit_transform(corpus).toarray()

In [82]:
x.shape

(1000, 5000)

In [83]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [121]:
msg

,level_0,index,Unnamed: 0,title,text,label
0,0,1821,9818,Hillary Ally Offers Strategy On How To Make Ne...,Trump To Host Facebook Live Nightly Show Until...,FAKE
1,1,1957,5121,Picking Mike Pence Really Was A Grand Slam For...,Mike Pence? Full disclosure: I served as head ...,REAL
2,2,4249,9370,4 Times As Many Americans Think Biased U.S. Me...,"Posted on November 1, 2016 by WashingtonsBlog ...",FAKE
3,3,708,3897,Secret Service testing ways to intercept rogue...,"Mysterious, middle-of-the-night drone flights ...",REAL
4,4,1894,1937,Rubio nabs key former Romney aide,Sen. Marco Rubio (R-Fla.) is adding a veteran ...,REAL
...,...,...,...,...,...,...
995,995,4854,2801,Administration officials defend Iran nuclear deal,WASHINGTON—Top administration officials forcef...,REAL
996,996,4341,3477,Who will fight for gay marriage at SCOTUS?,Washington (CNN) The date for arguments is on ...,REAL
997,997,4071,4626,How Gary Johnson's Vote Percentages Will Affec...,There has been a lot of weight placed in Liber...,REAL
998,998,3900,1794,What advantages does Scott Walker offer?,"Every GOP presidential candidate, most especia...",REAL


In [125]:
y=msg.label.apply(lambda x: 1 if x=='REAL' else 0)
y.value_counts()

0    521
1    479
Name: label, dtype: int64

In [126]:
cv.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': 5000,
 'min_df': 1,
 'ngram_range': (1, 3),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [127]:
cv.get_feature_names()

['abandon',
 'abc',
 'abc news',
 'abedin',
 'abil',
 'abl',
 'abolish',
 'abort',
 'abroad',
 'absenc',
 'absent',
 'absente',
 'absolut',
 'absorb',
 'absurd',
 'abu',
 'abus',
 'academ',
 'academi',
 'acceler',
 'accept',
 'access',
 'access pipelin',
 'accid',
 'accident',
 'accommod',
 'accompani',
 'accomplish',
 'accord',
 'accord new',
 'accord report',
 'account',
 'accur',
 'accus',
 'achiev',
 'acknowledg',
 'acquir',
 'across',
 'across board',
 'across countri',
 'act',
 'act like',
 'action',
 'activ',
 'activist',
 'actor',
 'actual',
 'acupunctur',
 'ad',
 'adam',
 'adapt',
 'add',
 'addict',
 'addit',
 'address',
 'adequ',
 'adher',
 'adjust',
 'adler',
 'admin',
 'administr',
 'administr offici',
 'admir',
 'admit',
 'adopt',
 'adult',
 'advanc',
 'advantag',
 'adversari',
 'advertis',
 'advic',
 'advis',
 'advisor',
 'advisori',
 'advoc',
 'advocaci',
 'affair',
 'affect',
 'affili',
 'affirm',
 'afford',
 'afford care',
 'afford care act',
 'afghan',
 'afghanistan',

In [128]:
from sklearn.model_selection import train_test_split

In [129]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [130]:
count_df= pd.DataFrame(x_train,columns=cv.get_feature_names())

In [131]:
count_df.head()

,abandon,abc,abc news,abedin,abil,abl,abolish,abort,abroad,absenc,...,young,young peopl,younger,youth,youtub,zealand,zelaya,zero,zika,zone
0,0.0,0.0,0.0,0.0,0.0,0.0157,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047021
1,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,...,0.070428,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.094897,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,...,0.047651,0.067138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [132]:
# Using Naive Bayes Algorithm

from sklearn.naive_bayes import MultinomialNB

In [133]:
classifier= MultinomialNB()

In [134]:
import itertools

In [135]:
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score

In [136]:
classifier.fit(x_train,y_train)

MultinomialNB()

In [137]:
pred= classifier.predict(x_test)

In [138]:
pred

array([1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1], dtype=int64)

In [139]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.91      0.90      0.90       111
           1       0.88      0.89      0.88        89

    accuracy                           0.90       200
   macro avg       0.89      0.89      0.89       200
weighted avg       0.90      0.90      0.90       200



In [140]:
confusion_matrix(y_test,pred)

array([[100,  11],
       [ 10,  79]], dtype=int64)

In [141]:
accuracy_score(y_test,pred)

0.895

In [142]:
# Passive Aggressive Classifier

from sklearn.linear_model import PassiveAggressiveClassifier

In [143]:
pac= PassiveAggressiveClassifier(max_iter=50)

In [144]:
pac.fit(x_train,y_train)

PassiveAggressiveClassifier(max_iter=50)

In [145]:
pred1= pac.predict(x_test)

In [146]:
pred1

array([1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1], dtype=int64)

In [147]:
print(classification_report(y_test,pred1))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89       111
           1       0.87      0.84      0.86        89

    accuracy                           0.88       200
   macro avg       0.87      0.87      0.87       200
weighted avg       0.87      0.88      0.87       200



In [148]:
confusion_matrix(y_test,pred1)

array([[100,  11],
       [ 14,  75]], dtype=int64)

In [149]:
accuracy_score(y_test,pred1)

0.875

In [150]:
classifier= MultinomialNB(alpha=0.1)

In [151]:
prev_score=0
for alpha in np.arange(0,1,0.1):
    sub_classifier=MultinomialNB(alpha=alpha)
    sub_classifier.fit(x_train,y_train)
    y_pred=sub_classifier.predict(x_test)
    score=accuracy_score(y_test,y_pred)
    if score>prev_score:
        classifier=sub_classifier
    print('Aplha : {}, Score : {}'.format(alpha,score))

c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\sklearn\naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


Aplha : 0.0, Score : 0.845
Aplha : 0.1, Score : 0.87
Aplha : 0.2, Score : 0.865
Aplha : 0.30000000000000004, Score : 0.86
Aplha : 0.4, Score : 0.86
Aplha : 0.5, Score : 0.86
Aplha : 0.6000000000000001, Score : 0.875
Aplha : 0.7000000000000001, Score : 0.875
Aplha : 0.8, Score : 0.885
Aplha : 0.9, Score : 0.89
